## train

**step 1:** Import and download DistilBertForQuestionAnswering from transformers. And Set up the PyTorch environment and initialize the DataLoader.

In [3]:
from transformers import DistilBertForQuestionAnswering
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
# 设置GPU / CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') # 将模型移到被检测设备
model.to(device)
# 激活模型的训练模式
model.train()
# 初始化AdamW优化器的权重衰减
optim = AdamW(model.parameters(), lr=5e-5)
# 初始化训练数据的数据加载器
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

ImportError: 
DistilBertForQuestionAnswering requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


**step2**: Load data and start fine-tuning the model.

In [ ]:
for epoch in range(3):
# 设置模型为训练模式
    model.train()
    # 设置循环(我们对进度条使用tqdm)
    loop = tqdm(train_loader, leave=True) 
    for batch in loop:
# 初始化计算的梯度(从上一步) 
        optim.zero_grad()
# 提取训练所需的所有张量批次
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device) start_positions = batch['start_positions'].to(device) end_positions = batch['end_positions'].to(device)
        # 训练模型，返回输出
        outputs = model(input_ids, attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)
        # 提取损失
        loss = outputs[0]
        # 计算每个需要更新的参数的损失 loss.backward()
        # 更新参数
        optim.step()
# 在进度条上打印相关信息 
        loop.set_description(f'Epoch {epoch}') 
        loop.set_postfix(loss=loss.item())

**step 4**:  Save the model and tokenizer.

In [ ]:
model_path = 'models/distilbert-custom'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

The next time load the model, you can use the saved model directly.

In [ ]:
model = AutoModel.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

**step 6**: Compute accuracy:

In [ ]:
# switch model out of training mode
model.eval()
#val_sampler = SequentialSampler(val_dataset)
val_loader = DataLoader(val_dataset, batch_size=16)
acc = []
# initialize loop for progress bar
loop = tqdm(val_loader)
# loop through batches
for batch in loop:
    # we don't need to calculate gradients as we're not training
    with torch.no_grad():
        # pull batched items from loader
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_true = batch['start_positions'].to(device)
        end_true = batch['end_positions'].to(device)
        # make predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        # pull preds out
        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)
        # calculate accuracy for both and append to accuracy list
        acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
        acc.append(((end_pred == end_true).sum()/len(end_pred)).item())
# calculate average accuracy in total
acc = sum(acc)/len(acc)

**step 7**: bash script find the best parameter. .(python)

In [ ]:
# get the hyperparameter from bash script

epoches = int(sys.argv[1])
batch_size = int(sys.argv[2])
## ...
sys.exit(acc)